In [3]:
# drop_from_eunju_robust.py (노트북 셀에 그대로 실행)
from pathlib import Path
import pandas as pd

# 1) 드랍 대상(❌ + ⚠️ 모두)
DROP_ITEMS = {
    # 깻잎
    "깻잎순",
    # 브로콜리
    "브로콜리(일반)",
    # 시금치
    "섬초", "포항초", "기타",
    # 쑥갓
    "쑥갓(일반)",
    # 애호박
    "호박(일반)", "풋호박",
    # 양배추
    "저장양배추", "기타", "꼬꼬마", "수입",
    # 양상추
    "수입", "기타",
    # 오이
    "청풍", "일반", "가시오이", "노각", "기타",
    # 치커리
    "세러리(국산)",
}

# 2) 세부품목 컬럼 후보
CANDIDATE_COLS = ["품목명", "세부 품목", "세부품목", "품목"]

# 3) 루트 폴더 자동 탐색 (현재 폴더 우선)
def pick_root():
    cwd = Path.cwd().resolve()
    candidates = [
        cwd,                               # 현재 폴더(노트북이 04 은주 안에 있으면 OK)
        cwd / "04 은주",                    # 현재 폴더 하위
        cwd.parent / "04 은주",            # 현재 폴더의 상위에 위치
        Path.home() / "Documents/GitHub/GDF_Final_G3/sojin/04 은주",  # 고정 경로 백업
    ]
    for p in candidates:
        if p.exists() and p.is_dir():
            # CSV가 실제 존재하는지까지 확인
            if list(p.glob("*.csv")):
                return p.resolve()
    # 그래도 못 찾으면 전체 트리에서 한 번 검색
    hits = list(cwd.glob("**/04 은주"))
    for h in hits:
        if list(h.glob("*.csv")):
            return h.resolve()
    raise SystemExit(f"[에러] '04 은주' 폴더(또는 그 안의 CSV)를 찾지 못했습니다. CWD={cwd}")

root = pick_root()
print(f"[INFO] 작업 폴더: {root}")

# 4) CSV 수집(이미 *_cleaned.csv 는 제외)
files = [p for p in sorted(root.glob("*.csv")) if not p.name.endswith("_cleaned.csv")]
print(f"[INFO] 발견한 CSV: {len(files)}개")
if not files:
    raise SystemExit("[중지] 처리할 CSV가 없습니다.")

total_before = total_after = 0

for fp in files:
    # 5) 읽기 (인코딩 자동)
    try:
        df = pd.read_csv(fp, encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(fp, encoding="cp949")

    # 6) 공백 정리(칼럼/문자열 값)
    df.columns = df.columns.str.strip()
    for c in df.select_dtypes(include="object").columns:
        df[c] = df[c].astype(str).str.strip()

    # 7) 세부품목 컬럼 탐지
    target_col = next((c for c in CANDIDATE_COLS if c in df.columns), None)
    if target_col is None:
        print(f"[SKIP] {fp.name}: 세부품목 컬럼을 찾지 못함 -> 보유 칼럼: {list(df.columns)[:10]}")
        continue

    before = len(df)
    present = sorted(set(df[target_col]) & DROP_ITEMS)
    df2 = df[~df[target_col].isin(DROP_ITEMS)].copy()
    after = len(df2)

    # 8) 저장 (원본 폴더 바로 아래 cleaned/)
    out_dir = fp.parent / "cleaned"
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"{fp.stem}_cleaned.csv"
    df2.to_csv(out_path, index=False, encoding="utf-8-sig")

    print(f"[OK] {fp.name}: {before} → {after} (드랍 {before-after}행) | 컬럼: {target_col} "
          f"| 제거된 세부품목: {present or '-'} | 저장: {out_path.name}")

    total_before += before
    total_after += after

print(f"\n[요약] {len(files)}개 처리 완료. 총 {total_before}행 → {total_after}행")
print(f"[완료] 결과는 '{root}/cleaned' 폴더에서 확인하세요.")


[INFO] 작업 폴더: /Users/sojinjung/Documents/GitHub/GDF_Final_G3/sojin/04 은주
[INFO] 발견한 CSV: 13개
[OK] 부추.csv: 10962 → 7308 (드랍 3654행) | 컬럼: 품목명 | 제거된 세부품목: ['기타'] | 저장: 부추_cleaned.csv
[OK] 깻잎.csv: 10962 → 3654 (드랍 7308행) | 컬럼: 품목명 | 제거된 세부품목: ['기타', '깻잎순'] | 저장: 깻잎_cleaned.csv
[OK] 미나리.csv: 7308 → 7308 (드랍 0행) | 컬럼: 품목명 | 제거된 세부품목: - | 저장: 미나리_cleaned.csv
[OK] 브로콜리.csv: 10962 → 10962 (드랍 0행) | 컬럼: 품목명 | 제거된 세부품목: - | 저장: 브로콜리_cleaned.csv
[OK] 생강.csv: 21924 → 18270 (드랍 3654행) | 컬럼: 품목명 | 제거된 세부품목: ['기타'] | 저장: 생강_cleaned.csv
[OK] 시금치.csv: 14616 → 3654 (드랍 10962행) | 컬럼: 품목명 | 제거된 세부품목: ['기타', '섬초', '포항초'] | 저장: 시금치_cleaned.csv
[OK] 쑥갓.csv: 3070 → 0 (드랍 3070행) | 컬럼: 품목명 | 제거된 세부품목: ['쑥갓(일반)'] | 저장: 쑥갓_cleaned.csv
[OK] 애호박.csv: 14616 → 7308 (드랍 7308행) | 컬럼: 품목명 | 제거된 세부품목: ['풋호박', '호박(일반)'] | 저장: 애호박_cleaned.csv
[OK] 양배추.csv: 18270 → 7308 (드랍 10962행) | 컬럼: 품목명 | 제거된 세부품목: ['기타', '꼬꼬마', '저장양배추'] | 저장: 양배추_cleaned.csv
[OK] 양상추.csv: 10962 → 7308 (드랍 3654행) | 컬럼: 품목명 | 제거된 세부품목: ['기타'] | 저장